### Домашнее задание по теме «Рекомендации на основе содержания»

Построить рекомендации (регрессия, предсказываем оценку) на фичах: TF-IDF на тегах и жанрах   

Средние оценки (+ median, variance, etc.) пользователя и фильма   

Оценить RMSE на тестовой выборке

In [254]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

%matplotlib inline

Заугрузим данные в датафреймы

In [487]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [256]:
movies.sample(3)

,movieId,title,genres
3672,5059,Little Dieter Needs to Fly (1997),Documentary
8240,104272,Blackfish (2013),Documentary
830,1091,Weekend at Bernie's (1989),Comedy


Очистим признак жанры от служебных симоволов и разделим пробелами

In [488]:
movies['genres'] = movies.genres.apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|')))

In [489]:
movies.sample(3)

,movieId,title,genres
438,502,"Next Karate Kid, The (1994)",Action Children Romance
2491,3325,"Next Best Thing, The (2000)",Comedy Drama
3470,4734,Jay and Silent Bob Strike Back (2001),Adventure Comedy


Очистим признак тэги от служебных симоволов и разделим пробелами

In [490]:
tags['tag'] = tags.tag.apply(lambda x: ' '.join(x.replace(' ', ' ').replace('-', '').split('|')))

In [497]:
tags.sample(3)

,userId,movieId,tag,timestamp
2882,537,105504,suspense,1424141149
1285,474,1088,music,1138306856
1163,474,647,Gulf War,1137375718


Объединим датафремы

In [474]:
movie_tags_merge = movies.merge(tags, how='left', on='movieId').fillna('0')
print(movie_tags_merge.info())
movie_tags_merge.sample(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11853 entries, 0 to 11852
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movieId    11853 non-null  int64 
 1   title      11853 non-null  object
 2   genres     11853 non-null  object
 3   userId     11853 non-null  object
 4   tag        11853 non-null  object
 5   timestamp  11853 non-null  object
dtypes: int64(1), object(5)
memory usage: 648.2+ KB
None


,movieId,title,genres,userId,tag,timestamp
760,476,"Inkwell, The (1994)",Comedy Drama,0,0,0
11480,165347,Jack Reacher: Never Go Back (2016),Action Crime Drama Mystery Thriller,0,0,0
5095,5633,Heaven (2002),Drama,474,bombs,1.13804e+09


In [413]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [415]:
movie_ratings_merge = movie_tags_merge.merge(ratings, how='left', on='movieId').fillna(0)

In [420]:
print(movie_ratings_merge.info())
movie_ratings_merge.sample(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285783 entries, 0 to 285782
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      285783 non-null  int64  
 1   title        285783 non-null  object 
 2   genres       285783 non-null  object 
 3   userId_x     285783 non-null  object 
 4   tag          285783 non-null  object 
 5   timestamp_x  285783 non-null  object 
 6   userId_y     285783 non-null  float64
 7   rating       285783 non-null  float64
 8   timestamp_y  285783 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 21.8+ MB
None


,movieId,title,genres,userId_x,tag,timestamp_x,userId_y,rating,timestamp_y
256232,66934,Dr. Horrible's Sing-Along Blog (2008),Comedy Drama Musical SciFi,477,Neil Patrick Harris,1.24479e+09,177.0,4.0,1.435525e+09
129424,1219,Psycho (1960),Crime Horror,477,Alfred Hitchcock,1.24258e+09,367.0,4.0,9.978113e+08
188999,3527,Predator (1987),Action SciFi Thriller,477,Jesse Ventura,1.26983e+09,95.0,4.5,1.105401e+09


Получим итоговый датафрейм с данными 

In [500]:
movies_result = movie_ratings_merge[['movieId', 'genres', 'tag', 'rating']]
movies_result.sample(3)
movies_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285783 entries, 0 to 285782
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  285783 non-null  int64  
 1   genres   285783 non-null  object 
 2   tag      285783 non-null  object 
 3   rating   285783 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.9+ MB


Подготовим данные для обучения модели

In [461]:
X = (movies_result['genres'] + ' ' + movies_result['tag']).values
y = movies_result['rating'].values

Преобразуем данные о жанрах и тэгах в векторы и обучим модель Линейной регрессии

In [465]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

In [466]:
X_train, X_test, y_train, y_test= train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [467]:
lr = LinearRegression().fit(X_train, y_train)

In [468]:
y_pred = lr.predict(X_test)

In [469]:
mean_squared_error(y_test, y_pred, squared=False)

0.9602211162551327

Добавим колонку с предсказаниями в наш итоговый датафрейм

In [502]:
prediction_rating = lr.predict(X_tfidf)

In [503]:
movies_result['Prediction'] = prediction_rating

<ipython-input-503-30c9251c3883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_result['Prediction'] = prediction_rating


In [507]:
movies_result.sample(10)

,movieId,genres,tag,rating,Prediction
246974,49530,Action Adventure Crime Drama Thriller War,Jennifer Connelly,4.0,3.681821
9605,110,Action Drama War,Medieval,4.0,4.018135
266175,79132,Action Crime Drama Mystery SciFi Thriller IMAX,complicated,5.0,4.149748
47397,296,Comedy Crime Drama Thriller,ensemble cast,4.5,4.176957
4812,39,Comedy Romance,chick flick,3.0,3.069541
2330,25,Drama Romance,alcoholism,2.0,3.700549
107769,780,Action Adventure SciFi Thriller,aliens,4.0,3.709327
113780,924,Adventure Drama SciFi,revolutionary,4.5,3.951612
54707,296,Comedy Crime Drama Thriller,Harvey Keitel,5.0,4.141394
57984,296,Comedy Crime Drama Thriller,intellectual,4.0,4.143521


#### Обучим модель Линейной регрессии на средних оценках пользователей